# Test: Bestimmung Precision / Recall mit SVM
Um die Klassifikation, und damit auch von dem Aufbau unserer Matrix, zu testen, bestimmen wir Precision und Recall mit einer SVM.

* Precision: Wieviele der zugeordneten Samples sind tatsächlich relevant? (true pos.) / (true pos. + false pos.)
* Recall: Wieviele der tatsächlich relevanten Samples wurden gefunden / zugeordnet? (true pos.) / (true pos. + false neg.)
 * Tiefe Precision / hoher Recall: Es werden viele Samples zugeordnet, von denen jedoch viele falsch sind
 * Hohe Precision / tiefer Recall: Es werden nur wenige Samples zugeordnet, von denen jedoch die meisten stimmen.
<img src="img/Precisionrecall.png" alt="Drawing" style="width: 200px"/>

## Optimierung
Mit diesem Test versuchen wir die Konfiguration der SVM möglichst gut auf unsere Datenstruktur auszurichten.
### Kernel
Der Kernel definiert, mit welcher Art Funktion die Datenpunkte im 2D-Raum voneinander getrennt werden. Standardmässig wird dazu eine lineare Funktion (linear kernel) verwendet. Wenn die Punkte im training set nicht durch eine Gerade trennbar sind, versucht man sie in einer höheren Dimension voneinander zu trennen. Die erreicht man durch andere kernel-Funktionen (z.B. Polynomial oder RBF).
Bei vielen Features, wie es bei unseren Daten der Fall ist, bringt eine höhere Dimension keine Vorteile, ist aber sehr teuer. Entsprechend verwenden wir einen linearen Kernel.

#### Parameter
Der lineare Kernel verlangt ausschliesslich einen Penalty Parameter C. Beim einem hohen C versucht der Classifier die Ebene so zu legen, dass möglichst wenige Datenpunkte aus dem training set falsch klassifiziert sind. Dafür nimmt er ein kleineres Margin zwischen Ebene und Punkte in Kauf. Bei einem tiefen Wert für C sucht der Classifier nach einem möglichst hohen margin, nimmt aber falsche Klassifizierungen im trianing set in Kauf.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from imports.matrix_helper import MatrixHelper
from imports.prediction_helper import PredictionHelper
from sklearn.metrics import precision_recall_curve

matrix_helper = MatrixHelper()

for filename in ['defs_current', 'defs_history', 'calls_current', 'calls_hisotry']:
    # Read pickle
    matrices = matrix_helper.load_from_parse('data/matrices/matrix_cla_' + filename + '.pickle')

    # Instantiate Prediction Helper Class and predict values for compare matrix with an SVM
    prediction_helper = PredictionHelper()
    prediction_helper.calculate_validation_compare_matrix(matrices, sampling_factor=(2.0/3), prediction_type='SVM', crop_matrix=False)
    compare_matrix = prediction_helper.get_compare_matrix()

    # Compute Precision-Recall
    precision, recall, thresholds = precision_recall_curve(np.array(compare_matrix[:, 2], dtype='f'), np.array(compare_matrix[:, 1], dtype='f'))

    # Print
    vulnerable_testset = matrix_helper.get_vulnerable_percentage(np.array(compare_matrix[:, 2], dtype='f'))
    vulnerable_prediction = matrix_helper.get_vulnerable_percentage(np.array(compare_matrix[:, 1], dtype='f'))
    print('|{:37}|{:^11.3f}|{:^8.3f}|{:16.1f}%|{:19.1f}%|({:5.1f}min)'.format(filename, precision[1], recall[1], vulnerable_testset, vulnerable_prediction, prediction_helper.time))

|defs_current                         |   0.679   | 0.068  |             9.8%|                1.0%|(108.9min)
|defs_history                         |   0.594   | 0.264  |             5.0%|                2.2%|( 49.8min)


## Ergebnisse
### Bestimmung Parameter C
Evaluation mit 2000 samples im test set:
* C=0.1: Precision 0.777777777778, Recall 0.21875
* C=0.2: Precision 0.785714285714, Recall 0.34375
* C=0.3: Precision 0.769230769231, Recall 0.3125
* C=0.4: Precision 0.714285714286, Recall 0.3125
* C=0.5: Precision 0.666666666667, Recall 0.3125
* C=0.6: Precision 0.555555555556, Recall 0.3125
* C=0.7: Precision 0.555555555556, Recall 0.3125
* C=0.8: Precision 0.578947368421, Recall 0.34375
* C=0.9: Precision 0.578947368421, Recall 0.34375
* C=1: Precision 0.578947368421, Recall 0.34375

### Test mit Parameter C
#### Test mit Klassifikationsmatrix nach dem Bugfix
* #1: Precision 0.814313346228, Recall 0.695867768595
* #2: Precision 0.796541200407, Recall 0.647107438017
* #3: Precision 0.804263565891, Recall 0.685950413223
* #4: Precision 0.819875776398, Recall 0.654545454545
* #5: Precision 0.802040816327, Recall 0.64958677686
* #6: Precision 0.828838174274, Recall 0.660330578512
* #7: Precision 0.807453416149, Recall 0.644628099174
* #8: Precision 0.805954825462, Recall 0.648760330579
* #9: Precision 0.800401203611, Recall 0.659504132231
* #10: Precision 0.812437311936, Recall 0.669421487603
* #11: Precision 0.789579158317, Recall 0.651239669421
* #12: Precision 0.810568295115, Recall 0.671900826446
* #13: Precision 0.80971659919, Recall 0.661157024793
* #14: Precision 0.821538461538, Recall 0.661983471074
* #15: Precision 0.805555555556, Recall 0.671074380165
* 
* Precision mean: 0.808605180427
* Recall mean: 0.662203856749

## Ergebnisse mit div Features
| Matrix (Classification)             | Precision | Recall | Vul. in Testset | Vul. in Prediction |
|-------------------------------------|-----------|--------|-----------------|--------------------|
| Includes (current)                  | 0.786     | 0.342  | 9.8%            | 4.3%               |
| Includes (history, with duplicates) | 0.8       | 0.661  | 17.8%           | 14,7%              |
| Includes (history)                  | 0.783     | 0.631  | 9.6%            | 7.7%               |
| Conditions (current)                | 0.794     | 0.097  | 9.8%            | 1.2%               |
| Conditions (history, with duplicates)| 0.88      | 0.517  | 17.4%           | 10.2%              |
| Conditions (history)                | 0.892     | 0.397  | 5.9%            | 2.6%               |
| Names (current)                     | 0.75      | 0.038  | 9.8%            | 0.5%               |
| Names (history, with duplicates)    | 0.671     | 0.483  | 13.5%           | 9.7%               |
| Names (history)                     | 0.556     | 0.126  | 2.1%            | 0.5%               |
| Defs (current)                      |   0.679   | 0.068  |             9.8%|                1.0%|
| Defs (history)                      |   0.594   | 0.264  |             5.0%|                2.2%|
